# Transformer Chatbot


## Data Loading



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
corpus = '/content/drive/My Drive/ToBigs/chatbot_for_the_disabled/Phase1 : Data Collection & Preprocessing /Data/통합데이터/final_Khaiii.pickle'
df = pd.read_pickle(corpus)
df.head()

,Question,Answer,Q_token,A_token,len_Q_token,len_A_token
0,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법제2조에 따른 학교에 재학중인 경우에...,"[중증/NNG, 장애/NNG, 아동수당/NNG, 을/JKO, 받/VV, 는/ETM,...","[중증/NNG, 장애/NNG, 아동수당/NNG, 을/JKO, 받/VV, 는/ETM,...",50,133
1,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요,장애인활동지원 홈페이지에서 지정된 활동지원기관 및 활동지원사 교육기관을 확인 할 수...,"[장애인활동/NNG, 지원/NNG, 교육/NNG, 기관/NNG, 과/JC, 제공/N...","[장애인활동/NNG, 지원/NNG, 홈페이지/NNG, 에서/JKB, 지정/NNG, ...",41,51
2,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요,장애인연금법 제6조제5항에 따라 기초연금 수급권자에게는 기초급여를 지급 하지 않습니다,"[기초/NNG, 연금/NNG, 수급/NNG, 권/XSN, 을/JKO, 포기/NNG,...","[장애인연금법/NNG, 제/XPN, 6/SN, 조제/NNG, 5/SN, 항/NNG,...",35,47
3,만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며 미리 신청하더라도 3...,"[만/NNG, 19/SN, 세/NNB, 중/NNB, 증장애인/NNG, 이/JKS, ...","[장애인연금법/NNG, 에/JKB, 따르/VV, 아/EC, 2/SN, 월/NNB, ...",54,59
4,장애인연금은 누가 받을 수 있나요,만18세 이상의 등록 장애인 중 장애인연금법상 중증장애인종전 12급 및 3급 중복 ...,"[장애인연금/NNG, 은/JX, 누구/NP, 가/JKS, 받/VV, 을/ETM, 수...","[만/NNG, 18/SN, 세/NNB, 이상/NNG, 의/JKG, 등록/NNG, 장...",18,69


In [0]:
import pandas as pd
import re

# Khaiii 토큰
# + 제거하고 pos정보 삭제, 토큰 구분은 쉼표로
# def tokenizing(x):
#     return ' '.join([''.join([j[:j.index('/')] for j in i.split('+')]).replace(' ', '') for i in x])

# + 제거하고 pos정보 삭제, 토큰 구분은 형태소로
# def tokenizing(x):
#     return ' '.join([' '.join([j[:j.index('/')].replace(' ','') for j in i.split('+')]) for i in x])

# 리스트 형태를 String으로 만들어줌
def tokenizing(x):
    return ' '.join(x)

def test(x):
    x = x.replace('/','+')
    x = x.replace('NNG','체언')
    return x

questions = df['Q_token'].apply(tokenizing) 
answers = df['A_token'].apply(tokenizing) 

questions = questions.apply(test)

print('Sample question: {}'.format(questions[4]))
print('Sample answer: {}'.format(answers[4]))


Sample question: 장애인연금+체언 은+JX 누구+NP 가+JKS 받+VV 을+ETM 수+NNB 있+VV 나요+EC
Sample answer: 만/NNG 18/SN 세/NNB 이상/NNG 의/JKG 등록/NNG 장애인/NNG 중/NNB 장애인/NNG 연금/NNG 법/NNG 상/XSN 중증장애/NNG 인종전/NNG 12/SN 급/NNG 및/MAG 3/SN 급/NNG 중복/NNG 장애인/NNG 에/JKB 해당/NNG 하/XSV 는/ETM 자/NNB 가/JKS 신청/NNG 하/XSV ㄹ/ETM 수/NNB 있/VV 습니다/EC


## TF model

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception as err:
    print(str(err))
    pass

# !pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
tf.random.set_seed(1234)

!pip install tensorflow-datasets==1.2.0
import tensorflow_datasets as tfds

import os
import numpy as np

import matplotlib.pyplot as plt

TensorFlow 2.x selected.
     |████████████████████████████████| 2.3MB 6.5MB/s 
  Found existing installation: tensorflow-datasets 1.3.0
    Uninstalling tensorflow-datasets-1.3.0:
      Successfully uninstalled tensorflow-datasets-1.3.0


In [0]:
print(f'tf version : {tf.__version__}')

device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
    print('GPU device not found')
print(f'GPU at : {device}')

tf version : 2.0.0
GPU at : /device:GPU:0


In [0]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor in questions:
  tokens = tokenizer.tokenize(text_tensor)
  vocabulary_set.update(tokens)

vocab_size = len(vocabulary_set)
vocab_size

63264

In [0]:
# Build tokenizer using tfds for both questions and answers
# 함수 하나로 간단하게 vocab을 구축함

# tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
#     questions + answers, target_vocab_size=2**13)

tokenizer = tfds.features.text.TokenTextEncoder(vocabulary_set)


# 스타트 토큰과 엔드 토큰을 만듦
# vocab에는 {'장애인'(단어) : 128(번호)} 인덱싱이 되어있는데
# SOS와 EOS는 그냥 번호만 있고 vocab에는 안들어있음 패딩토큰도 마찬가지. 일부러 그런건가?
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# Vocabulary size plus start and end token
VOCAB_SIZE = tokenizer.vocab_size + 2

In [0]:
encoded_example = tokenizer.encode(questions[20])
print(f'Tokenized sample question: {encoded_example}')
print(f'Tokenized sample question: {tokenizer.decode(encoded_example)}')

for ts in encoded_example:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

Tokenized sample question: [60769, 30065, 8484, 24786, 13247, 11993, 32545, 24786, 43393, 2382, 39782, 49247, 35565, 24786, 60769, 24786, 37126, 24786, 48216, 11993, 43867, 36536, 264, 15857, 43393, 47316, 8252, 15857]
Tokenized sample question: 장애 NNP 수당 체언 을 JKO 신청 체언 하 XSV 는 ETM 경우 체언 장애 체언 재심사 체언 를 JKO 받 VV 아야 EC 하 VX 나요 EC
60769 ----> 장애
30065 ----> NNP
8484 ----> 수당
24786 ----> 체언
13247 ----> 을
11993 ----> JKO
32545 ----> 신청
24786 ----> 체언
43393 ----> 하
2382 ----> XSV
39782 ----> 는
49247 ----> ETM
35565 ----> 경우
24786 ----> 체언
60769 ----> 장애
24786 ----> 체언
37126 ----> 재심사
24786 ----> 체언
48216 ----> 를
11993 ----> JKO
43867 ----> 받
36536 ----> VV
264 ----> 아야
15857 ----> EC
43393 ----> 하
47316 ----> VX
8252 ----> 나요
15857 ----> EC


In [0]:
# Maximum sentence length
MAX_LENGTH = 30


# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
    # check tokenized sentence max length
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)

In [0]:
print('Vocab size: {}'.format(VOCAB_SIZE))
print('Number of samples: {}'.format(len(questions)))

Vocab size: 63268
Number of samples: 479


In [0]:
questions

array([[63266,  4079, 24786, ...,     0,     0,     0],
       [63266, 10736, 24786, ...,     0,     0,     0],
       [63266, 58052, 24786, ...,     0,     0,     0],
       ...,
       [63266,  2553, 24786, ...,  8252, 15857, 63267],
       [63266, 38350, 24786, ...,     0,     0,     0],
       [63266, 50547, 24786, ..., 63267,     0,     0]], dtype=int32)

In [0]:
BATCH_SIZE = 32
BUFFER_SIZE = 20000

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

* inputs : 인풋 그대로
* 만약 answer = 'Start 날 조커라고 소개해줄래요? End' 
    * dec_inputs : 'Start 날 조커라고 소개해줄래요?'
    * outputs : '날 조커라고 소개해줄래요? End'
* 마치 Seq2Seq의 디코더 부분에서 디코더의 인풋과 아웃풋을 생각하면 좋을듯

## Atteniton

### Scaled dot product Attention

The scaled dot-product attention function used by the transformer takes three inputs: Q (query), K (key), V (value). The equation used to calculate the attention weights is:

$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

As the softmax normalization is done on the `key`, its values decide the amount of importance given to the `query`.

The output represents the multiplication of the attention weights and the `value` vector. This ensures that the words we want to focus on are kept as is and the irrelevant words are flushed out.

The dot-product attention is scaled by a factor of square root of the depth. This is done because for large values of depth, the dot product grows large in magnitude pushing the softmax function where it has small gradients resulting in a very hard softmax. 

For example, consider that `query` and `key` have a mean of 0 and variance of 1. Their matrix multiplication will have a mean of 0 and variance of `dk`. Hence, *square root of `dk`* is used for scaling (and not any other number) because the matmul of `query` and `key` should have a mean of 0 and variance of 1, so that we get a gentler softmax.

The mask is multiplied with *-1e9 (close to negative infinity).* This is done because the mask is summed with the scaled matrix multiplication of `query` and `key` and is applied immediately before a softmax. The goal is to zero out these cells, and large negative inputs to softmax are near zero in the output.

In [0]:
def scaled_dot_product_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # scale matmul_qk
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth) # scaling 

  # add the mask to zero out padding tokens
  if mask is not None:
    logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k)
  attention_weights = tf.nn.softmax(logits, axis=-1) 

  output = tf.matmul(attention_weights, value)

  return output

In [0]:
# temp_q = [[.1,.3,.3,.1]]
# temp_k = [[.1,.3,.3,.1], [.1,.3,.3,.1], [.1,.3,.3,.1]]
# # # scaled_dot_product_attention(temp_q, temp_k, temp_v, mask=None)
# matmul_qk = tf.matmul(temp_q, temp_k, transpose_b=True)
# print(matmul_qk)
# print(matmul_qk.shape)

# depth = tf.cast(tf.shape(temp_k)[-1], tf.float32) # (1, 4)
# print(depth)

# # scaled score
# # logits 

tf.Tensor([[0.2 0.2 0.2]], shape=(1, 3), dtype=float32)
(1, 3)
tf.Tensor(4.0, shape=(), dtype=float32)


### Multi-head attention

<img src="https://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png" width="500" alt="multi-head attention">


Multi-head attention consists of four parts:
* Linear layers and split into heads.
* Scaled dot-product attention.
* Concatenation of heads.
* Final linear layer.

Each multi-head attention block gets three inputs; Q (query), K (key), V (value). These are put through linear (Dense) layers and split up into multiple heads. 

The `scaled_dot_product_attention` defined above is applied to each head (broadcasted for efficiency). An appropriate mask must be used in the attention step.  The attention output for each head is then concatenated (using `tf.transpose`, and `tf.reshape`) and put through a final `Dense` layer.

Instead of one single attention head, `query`, `key`, and `value` are split into multiple heads because it allows the model to jointly attend to information at different positions from different representational spaces. After the split each head has a reduced dimensionality, so the total computation cost is the same as a single head attention with full dimensionality.

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs

## Transformer

### Masking



`create_padding_mask` and `create_look_ahead` are helper functions to creating masks to mask out padded tokens, we are going to use these helper functions as `tf.keras.layers.Lambda` layers.

Mask all the pad tokens (value `0`) in the batch to ensure the model does not treat padding as input.

In [0]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [0]:
print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

tf.Tensor(
[[[[0. 0. 1. 0. 1.]]]


 [[[1. 1. 1. 0. 0.]]]], shape=(2, 1, 1, 5), dtype=float32)


Look-ahead mask to mask the future tokens in a sequence.
We also mask out pad tokens.

i.e. To predict the third word, only the first and second word will be used

In [0]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [0]:
print(create_look_ahead_mask(tf.constant([[1, 2, 0, 4, 5]])))

tf.Tensor(
[[[[0. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 1. 0. 1.]
   [0. 0. 1. 0. 0.]]]], shape=(1, 1, 5, 5), dtype=float32)


### Positional encoding

Since this model doesn't contain any recurrence or convolution, positional encoding is added to give the model some information about the relative position of the words in the sentence. 

The positional encoding vector is added to the embedding vector. Embeddings represent a token in a d-dimensional space where tokens with similar meaning will be closer to each other. But the embeddings do not encode the relative position of words in a sentence. So after adding the positional encoding, words will be closer to each other based on the *similarity of their meaning and their position in the sentence*, in the d-dimensional space.

See the notebook on [positional encoding](https://github.com/tensorflow/examples/blob/master/community/en/position_encoding.ipynb) to learn more about it. The formula for calculating the positional encoding is as follows:

$$\Large{PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$

In [0]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [0]:
# sample_pos_encoding = PositionalEncoding(50, 512)

# plt.pcolormesh(sample_pos_encoding.pos_encoding.numpy()[0], cmap='RdBu')
# plt.xlabel('Depth')
# plt.xlim((0, 512))
# plt.ylabel('Position')
# plt.colorbar()
# plt.show()

### Encoder Layer

Each encoder layer consists of sublayers:

1. Multi-head attention (with padding mask) 
2. 2 dense layers followed by dropout

Each of these sublayers has a residual connection around it followed by a layer normalization. Residual connections help in avoiding the vanishing gradient problem in deep networks.

The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis.

In [0]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
# sample_encoder_layer = encoder_layer(
#     units=512,
#     d_model=128,
#     num_heads=4,
#     dropout=0.3,
#     name="sample_encoder_layer")

# tf.keras.utils.plot_model(
#     sample_encoder_layer, to_file='encoder_layer.png', show_shapes=True)

### Encoder

The Encoder consists of:
1.   Input Embedding
2.   Positional Encoding
3.   `num_layers` encoder layers

The input is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the encoder layers. The output of the encoder is the input to the decoder.

In [0]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
# sample_encoder = encoder(
#     vocab_size=8192,
#     num_layers=2,
#     units=512,
#     d_model=128,
#     num_heads=4,
#     dropout=0.3,
#     name="sample_encoder")

# tf.keras.utils.plot_model(
#    sample_encoder, to_file='encoder.png', show_shapes=True)

### Decoder Layer

Each decoder layer consists of sublayers:

1.   Masked multi-head attention (with look ahead mask and padding mask)
2.   Multi-head attention (with padding mask). `value` and `key` receive the *encoder output* as inputs. `query` receives the *output from the masked multi-head attention sublayer.*
3.   2 dense layers followed by dropout

Each of these sublayers has a residual connection around it followed by a layer normalization. The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis.

As `query` receives the output from decoder's first attention block, and `key` receives the encoder output, the attention weights represent the importance given to the decoder's input based on the encoder's output. In other words, the decoder predicts the next word by looking at the encoder output and self-attending to its own output. See the demonstration above in the scaled dot product attention section.

In [0]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [0]:
# sample_decoder_layer = decoder_layer(
#     units=512,
#     d_model=128,
#     num_heads=4,
#     dropout=0.3,
#     name="sample_decoder_layer")

# tf.keras.utils.plot_model(
#     sample_decoder_layer, to_file='decoder_layer.png', show_shapes=True)

### Decoder

The Decoder consists of:
1.   Output Embedding
2.   Positional Encoding
3.   N decoder layers

The target is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the decoder layers. The output of the decoder is the input to the final linear layer.

In [0]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [0]:
# sample_decoder = decoder(
#     vocab_size=8192,
#     num_layers=2,
#     units=512,
#     d_model=128,
#     num_heads=4,
#     dropout=0.3,
#     name="sample_decoder")

# tf.keras.utils.plot_model(
#     sample_decoder, to_file='decoder.png', show_shapes=True)

### Transformer

Transformer consists of the encoder, decoder and a final linear layer. The output of the decoder is the input to the linear layer and its output is returned.

In [0]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  # mask the future tokens for decoder inputs at the 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  # mask the encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [0]:
# sample_transformer = transformer(
#     vocab_size=8192,
#     num_layers=4,
#     units=512,
#     d_model=128,
#     num_heads=4,
#     dropout=0.3,
#     name="sample_transformer")

# tf.keras.utils.plot_model(
#     sample_transformer, to_file='transformer.png', show_shapes=True)

## Train model

### Initialize model

To keep this example small and relatively fast, the values for *num_layers, d_model, and units* have been reduced. See the [paper](https://arxiv.org/abs/1706.03762) for all the other versions of the transformer.

In [0]:
tf.keras.backend.clear_session()

# Hyper-parameters
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.3

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

### Loss function

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [0]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

### Custom learning rate

Use the Adam optimizer with a custom learning rate scheduler according to the formula in the [paper](https://arxiv.org/abs/1706.03762).

$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
# sample_learning_rate = CustomSchedule(d_model=128)

# plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
# plt.ylabel("Learning Rate")
# plt.xlabel("Train Step")

### Compile Model


In [0]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

### Fit model

Train our transformer by simply calling `model.fit()`

In [0]:
EPOCHS = 1

hist = model.fit(dataset, epochs=EPOCHS)

     16/Unknown - 19s 1s/step - loss: 6.9557 - accuracy: 0.0000e+00

CancelledError: ignored

## Evaluate and predict

The following steps are used for evaluation:

* Apply the same preprocessing method we used to create our dataset for the input sentence.
* Tokenize the input sentence and add `START_TOKEN` and `END_TOKEN`. 
* Calculate the padding masks and the look ahead masks.
* The decoder then outputs the predictions by looking at the encoder output and its own output.
* Select the last word and calculate the argmax of that.
* Concatentate the predicted word to the decoder input as pass it to the decoder.
* In this approach, the decoder predicts the next word based on the previous words it predicted.

Note: The model used here has less capacity and trained on a subset of the full dataset, hence its performance can be further improved.

In [0]:
def evaluate(sentence):
#   sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [0]:
sentence = '발달장애 이 ㄴ 주간 활동 서비스 신청 대상 은 어떻 게 되 나요'
sentence = tf.expand_dims(START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
sentence, output 

(<tf.Tensor: id=17370, shape=(1, 15), dtype=int32, numpy=
 array([[63266, 50143,   616, 58628, 54612,  9885, 16785, 32545,  5649,
         57006,   179, 12494, 41026,  8252, 63267]], dtype=int32)>,
 <tf.Tensor: id=17373, shape=(1, 1), dtype=int32, numpy=array([[63266]], dtype=int32)>)

In [0]:
output = tf.expand_dims(START_TOKEN, 0)
for i in range(MAX_LENGTH):
    # 3차원 텐서로 나옴
    predictions = model(inputs=[sentence, output], training=False)
    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # # concatenated the predicted_id to the output which is given to the decoder
    # # as its input.
    # 2차원 텐서
    output = tf.concat([output, predicted_id], axis=-1)

# 1차원텐서
prediction = tf.squeeze(output, axis=0)

In [0]:
# string
predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])
predicted_sentence

'구만 3200 중증장애인폭행 중증장애인폭행 경증장애수당인 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 3200 요바닥 3200 3200 3200 3200'

Let's test our model!

In [0]:
output = predict('발달장애 이 ㄴ 주간 활동 서비스 신청 대상 은 어떻 게 되 나요')

In [0]:
output = predict('발달장애인 주간 활동서비스 신청 대상은 어떻게 되나요')

### Khaiii 설치

* predict 단계에서 새로운 인풋을 받으면 kahiii로 토크나이징하고 모델에 넣어보려 함

In [0]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [0]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

test_input = '발달장애인 주간 활동서비스 신청 대상은 어떻게 되나요'

for word in api.analyze(test_input):
  morphs_str = " + ".join([(m.lex + '/' + m.tag) for m in word.morphs])


  # 수정중!